# 4 — Your First ROS2 Node

> Write, build, and run your first ROS2 Python node from scratch.

**Goal:** Understand the anatomy of a ROS2 Python node and see it run.

In [ ]:
#| default_exp first_node

## 4.1 The Minimal Node

Every ROS2 Python node follows this pattern:

1. **Import** `rclpy` and `Node`
2. **Subclass** `Node` (or use it directly)
3. **Initialize** ROS2, create the node, **spin** (process callbacks), **shutdown**

Here's the simplest possible node:

In [ ]:
#| export

MINIMAL_NODE = '''
#!/usr/bin/env python3
"""Minimal ROS2 node — just logs a message and spins."""

import rclpy                          # ROS2 Python client library
from rclpy.node import Node           # Base class for all nodes


class MinimalNode(Node):
    """A node that does nothing but exist and log a greeting."""

    def __init__(self):
        # Call the parent constructor with our node name
        super().__init__('minimal_node')   # <-- this name appears in `ros2 node list`
        self.get_logger().info('Hello from MinimalNode! I am alive.')


def main(args=None):
    """Entry point called by ROS2 (registered in setup.py)."""
    # 1. Initialize the ROS2 Python library
    rclpy.init(args=args)

    # 2. Create our node
    node = MinimalNode()

    # 3. Spin — keep the node alive, processing callbacks
    #    This blocks until Ctrl+C or shutdown
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        # 4. Cleanup
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(MINIMAL_NODE)

## 4.2 Line-by-Line Breakdown

Let's understand every line:

### `import rclpy`
This is the **ROS2 Client Library for Python** (rclpy). It wraps the C-level `rcl` library and gives you Python classes for nodes, publishers, subscribers, timers, etc.

### `class MinimalNode(Node)`
We **subclass `Node`**. This is the recommended OOP pattern in ROS2. The `Node` base class provides:
- `self.create_publisher(...)` — create a publisher
- `self.create_subscription(...)` — create a subscriber
- `self.create_timer(...)` — periodic callbacks
- `self.create_service(...)` — create a service server
- `self.get_logger()` — structured logging
- `self.declare_parameter(...)` — declare parameters

### `super().__init__('minimal_node')`
The node **name** is how this node is identified in the ROS2 graph. It shows up in:
- `ros2 node list`
- `rqt_graph`
- Log messages

### `rclpy.init(args=args)`
Initializes the ROS2 communication layer. Must be called **once** before creating any node.

### `rclpy.spin(node)`
Enters an **event loop** that:
1. Waits for messages on subscribed topics
2. Fires timer callbacks
3. Handles service requests

Without `spin()`, your callbacks never fire.

### `node.destroy_node()` + `rclpy.shutdown()`
Clean shutdown: releases DDS resources and lets the process exit cleanly.

## 4.3 Add a Timer — Periodic Work

Most nodes need to do something **repeatedly**. The simplest way is a **timer**:

In [ ]:
#| export

TIMER_NODE = '''
#!/usr/bin/env python3
"""A node with a timer that logs a counter every second."""

import rclpy
from rclpy.node import Node


class TimerNode(Node):
    def __init__(self):
        super().__init__('timer_node')
        
        # Create a timer: calls self.timer_callback every 1.0 second
        self.timer = self.create_timer(
            timer_period_sec=1.0,     # how often (seconds)
            callback=self.timer_callback
        )
        self.counter = 0
        self.get_logger().info('TimerNode started — will tick every 1 second')

    def timer_callback(self):
        """Called every 1.0 second by the timer."""
        self.counter += 1
        self.get_logger().info(f'Timer tick #{self.counter}')


def main(args=None):
    rclpy.init(args=args)
    node = TimerNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(TIMER_NODE)

### How Timers Work

```
rclpy.spin(node)
  │
  ├── wait 1.0s ──► timer_callback()  → "Timer tick #1"
  ├── wait 1.0s ──► timer_callback()  → "Timer tick #2"
  ├── wait 1.0s ──► timer_callback()  → "Timer tick #3"
  └── ... (until Ctrl+C)
```

The executor inside `spin()` checks if the timer period has elapsed and calls your function.

## 4.4 Add Parameters — Runtime Configuration

Let's make the timer period configurable:

In [ ]:
#| export

PARAM_NODE = '''
#!/usr/bin/env python3
"""A node with a configurable timer period parameter."""

import rclpy
from rclpy.node import Node


class ParameterTimerNode(Node):
    def __init__(self):
        super().__init__('param_timer_node')
        
        # Declare a parameter with a default value
        self.declare_parameter('timer_period', 1.0)
        self.declare_parameter('node_name', 'MyRobot')

        # Read the parameter value
        period = self.get_parameter('timer_period').value
        name = self.get_parameter('node_name').value

        self.timer = self.create_timer(period, self.timer_callback)
        self.counter = 0
        self.get_logger().info(
            f'{name} started — timer period: {period}s'
        )

    def timer_callback(self):
        self.counter += 1
        name = self.get_parameter('node_name').value
        self.get_logger().info(f'[{name}] tick #{self.counter}')


def main(args=None):
    rclpy.init(args=args)
    node = ParameterTimerNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(PARAM_NODE)

### Run with Custom Parameters

```bash
# Use default values
ros2 run my_robot_pkg param_timer_node

# Override from command line
ros2 run my_robot_pkg param_timer_node --ros-args \
  -p timer_period:=0.5 \
  -p node_name:=FastBot

# Or load from a YAML file
ros2 run my_robot_pkg param_timer_node --ros-args \
  --params-file config/params.yaml
```

## 4.5 Logging Levels

The ROS2 logger supports standard severity levels:

```python
self.get_logger().debug('Detailed debug info')    # Hidden by default
self.get_logger().info('Normal operation')         # Default visible
self.get_logger().warn('Something unexpected')     # Yellow in terminal
self.get_logger().error('Something went wrong')    # Red in terminal
self.get_logger().fatal('Unrecoverable error')     # Critical
```

Change log level at runtime:

```bash
# Set log level for a specific node
ros2 run my_robot_pkg timer_node --ros-args --log-level debug
```

## 4.6 Register in setup.py & Build

For ROS2 to find your node, register it in `setup.py`:

```python
# In setup.py
entry_points={
    'console_scripts': [
        'minimal_node = my_robot_pkg.minimal_node:main',
        'timer_node = my_robot_pkg.timer_node:main',
        'param_timer_node = my_robot_pkg.param_timer_node:main',
    ],
},
```

Then:

```bash
cd ~/ros2_ws
colcon build --symlink-install --packages-select my_robot_pkg
source install/setup.bash

# Run it!
ros2 run my_robot_pkg minimal_node
```

## 4.7 Inspect Your Running Node

While your node is running, open another terminal and inspect it:

```bash
# See your node in the list
ros2 node list
# /minimal_node

# Get detailed info
ros2 node info /minimal_node
# Shows:
#   /minimal_node
#     Subscribers: /parameter_events
#     Publishers:  /parameter_events, /rosout
#     Service Servers: .../describe_parameters, .../get_parameters, ...

# See log output from /rosout topic
ros2 topic echo /rosout
```

Notice that even the "minimal" node has built-in publishers and services for **parameter management** — ROS2 adds these automatically.

## 4.8 Node Lifecycle (Advanced Preview)

ROS2 supports **managed lifecycle nodes** for production systems:

```
Unconfigured → Inactive → Active → Finalized
                  ↑          │
                  └──────────┘  (deactivate/reactivate)
```

This lets an external manager (like Nav2's lifecycle manager) bring nodes up and down in a controlled order. We won't use lifecycle nodes in this course, but know they exist for serious robot deployments.

## 4.9 Exercises

Try these before moving to the next notebook:

1. **Modify the timer period** — Make the timer fire every 0.5 seconds
2. **Add a parameter** — Add a `greeting` parameter that changes the log message
3. **Multiple timers** — Create a node with two timers at different rates
4. **Use `ros2 param set`** — While the param_timer_node is running, change the `node_name` parameter from another terminal

---

**Next →** [Notebook 05: Topics — Publishers & Subscribers](05_topics_pubsub.ipynb)